In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pickle
from collections import defaultdict
import gc, sys

In [ ]:
class Category:
    def __init__(self, unique_items, item_to_id={}):
        self.items = unique_items
        self.item_to_id = item_to_id
        for _id, item_name in enumerate(self.items):
            self.item_to_id[item_name] = _id
        self.type = np.int16
        if len(self.items) > 30000:
            self.type = np.int32
    
    def encode_series(self, items):
        return np.array([self.item_to_id[_item] for _item in items]).astype(self.type)

In [ ]:
with open('../input/m5-point-forecast-createinput-1400/X_train_df', 'rb') as f:
    X_train_df = pickle.load(f)
y_train = np.load('../input/m5-point-forecast-createinput-1400/y_train.npy')

In [ ]:
with open('../input/m5-point-forecast-valweights/weights_df', 'rb') as f:
    val_weights = pickle.load(f)

In [ ]:
val_weights

In [ ]:
X_train_df['y'] = y_train

In [ ]:
train_df = X_train_df

In [ ]:
train_df = train_df.loc[train_df.day_id >= 1400].reset_index(drop=True)

In [ ]:
del X_train_df, y_train
gc.collect()

In [ ]:
train_df.shape

In [ ]:
with open('../input/m5-point-forecast-createinput-1400/cat_feats.pkl', 'rb') as f:
    cat_feats = pickle.load(f)
with open('../input/m5-point-forecast-createinput-1400/cat_feats_name2idx.pkl', 'rb') as f:
    cat_feats_name2idx = pickle.load(f)
with open('../input/m5-point-forecast-createinput-1400/cat_feats_objs.pkl', 'rb') as f:
    cat_feats_objs = pickle.load(f)

In [ ]:
col_names = train_df.columns

In [ ]:
col_names

In [ ]:
(val_weights.weight == 0).sum()

In [ ]:
# create historical features

In [ ]:
sales_train_val = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')

In [ ]:
last_day = 1941

In [ ]:
col_names_days = []
for i in range(1,last_day+1):
    col_names_days.append('d_'+str(i))

In [ ]:
# 28 day lag
_28dl_dict = {}
for i in range(len(sales_train_val)):
    _row = sales_train_val.iloc[i]
    unit_sales = _row.iloc[6:].values
    _lag = np.zeros(last_day) - 1
    _lag[-len(unit_sales):] = unit_sales
    _28dl_dict[(_row.item_id, _row.store_id)] = _lag.astype(np.float32)
    

In [ ]:
_ser = _28dl_dict[('HOBBIES_1_046', 'CA_1')]; _ser

In [ ]:
# 28_day_lag, 7_day_mean, 7_day_std, 28_day_mean, 28_day_std

_28dl_7dm_dict = {}
_28dl_7ds_dict = {}
_28dl_14dm_dict = {}
_28dl_14ds_dict = {}
_28dl_21dm_dict = {}
_28dl_21ds_dict = {}
_28dl_28dm_dict = {}
_28dl_28ds_dict = {}
_count=0
for itst, ser in _28dl_dict.items():
    if _count%1000 == 0:
        print(_count)
    _count+=1
    _28dl_7dm_dict[itst] = pd.Series(ser).rolling(7).mean().values.astype(np.float32)
    _28dl_7ds_dict[itst] = pd.Series(ser).rolling(7).std().values.astype(np.float32)
    _28dl_14dm_dict[itst] = pd.Series(ser).rolling(14).mean().values.astype(np.float32)
    _28dl_14ds_dict[itst] = pd.Series(ser).rolling(14).std().values.astype(np.float32)
    _28dl_21dm_dict[itst] = pd.Series(ser).rolling(21).mean().values.astype(np.float32)
    _28dl_21ds_dict[itst] = pd.Series(ser).rolling(21).std().values.astype(np.float32)
    _28dl_28dm_dict[itst] = pd.Series(ser).rolling(28).mean().values.astype(np.float32)
    _28dl_28ds_dict[itst] = pd.Series(ser).rolling(28).std().values.astype(np.float32)

In [ ]:
del sales_train_val
gc.collect()

In [ ]:
def show_progress(progress_str, n_cols=80):
    sys.stdout.write(progress_str + (" " * max(0, n_cols - len(progress_str))) + "\r")
    sys.stdout.flush()

In [ ]:
hist_feats = ['_28dl', '_28dl_7dm', '_28dl_7ds', '_28dl_14dm', '_28dl_14ds', '_28dl_21dm', '_28dl_21ds', '_28dl_28dm', '_28dl_28ds']

In [ ]:
def item_store_ordering(df_y):
    df_y_gpby_is = df_y.groupby(by=['item_id', 'store_id'])
    #result_df = pd.DataFrame(columns=col_names)
    result_dict = defaultdict(list)
    stats_dict = defaultdict(list)
    for i in range(len(val_weights)):
        if (i%1000) == 0:
            show_progress(f"Processing item_store: {i+1}/{len(val_weights)}")
        _row = val_weights.iloc[i]
        if _row.weight == 0:
            continue
        _item = cat_feats_objs[cat_feats_name2idx['item_id']].item_to_id[_row.item_id]
        _store = cat_feats_objs[cat_feats_name2idx['store_id']].item_to_id[_row.store_id]
        _df = df_y_gpby_is.get_group((_item, _store))
        result_dict['_28dl'].append(_28dl_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_7dm'].append(_28dl_7dm_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_7ds'].append(_28dl_7ds_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_14dm'].append(_28dl_14dm_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_14ds'].append(_28dl_14ds_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_21dm'].append(_28dl_21dm_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_21ds'].append(_28dl_21ds_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_28dm'].append(_28dl_28dm_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_28ds'].append(_28dl_28ds_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        #_df = _df.loc[(_df.day_id >= _row.day_of_first_sale).values]
        for col in col_names:
            if col == 'y':
                y = _df[col].values
                result_dict[col].append(y)
                # Update stats dict
                #stats_dict['item'].append(_item)
                #stats_dict['store'].append(_store)
                #stats_dict['y_norm_mean'].append(y_norm.mean())
                #stats_dict['y_norm_std'].append(y_norm.std())
                #stats_dict['y_norm_min'].append(y_norm.min())
                #stats_dict['y_norm_max'].append(y_norm.max())
            else:
                result_dict[col].append(_df[col].values)
    for col in col_names:
        result_dict[col] = np.concatenate(result_dict[col])
    for col in hist_feats:
        result_dict[col] = np.concatenate(result_dict[col])
    return pd.DataFrame(result_dict)[list(col_names)+hist_feats]

In [ ]:
ordered_train_df = item_store_ordering(train_df)
del train_df
gc.collect()

In [ ]:
ordered_train_df

In [ ]:
with open('../input/m5-point-forecast-createinput-1400/X_val_df', 'rb') as f:
    X_val_df = pickle.load(f)
y_val = np.load('../input/m5-point-forecast-createinput-1400/y_val.npy')

In [ ]:
X_val_df['y'] = y_val

In [ ]:
val_df = X_val_df

In [ ]:
ordered_val_df = item_store_ordering(val_df)
del val_df, X_val_df, y_val
gc.collect()

In [ ]:
ordered_train_df.to_pickle('ordered_train_df.pkl')

In [ ]:
ordered_val_df.to_pickle('ordered_val_df.pkl')

In [ ]:
val_weights.to_pickle('val_weights.pkl')

In [ ]:
! ls -alrh

In [ ]:
with open('_28dl_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_dict, f)

In [ ]:
with open('_28dl_7dm_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_7dm_dict, f)

In [ ]:
with open('_28dl_7ds_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_7ds_dict, f)

In [ ]:
with open('_28dl_14dm_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_14dm_dict, f)

In [ ]:
with open('_28dl_14ds_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_14ds_dict, f)

In [ ]:
with open('_28dl_21dm_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_7dm_dict, f)

In [ ]:
with open('_28dl_21ds_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_7ds_dict, f)

In [ ]:
with open('_28dl_28dm_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_28dm_dict, f)

In [ ]:
with open('_28dl_28ds_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_28ds_dict, f)

In [ ]:
! ls -alrh 

In [ ]:
with open('hist_feats.pkl', 'wb') as f:
    pickle.dump(hist_feats, f)

In [ ]:
! ls -alrh 